# Análise de Dados - Pipeline DataOps

Este notebook demonstra análise exploratória dos dados processados pelo pipeline ETL.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuração dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Carregamento dos Dados

In [ ]:
# Carregar dados processados
try:
    sales_df = pd.read_csv('../data/processed/sales_clean.csv')
    customers_df = pd.read_csv('../data/processed/customers_clean.csv')
    products_df = pd.read_csv('../data/processed/products_clean.csv')
    
    # Carregar resumos
    daily_summary = pd.read_csv('../data/processed/daily_summary.csv')
    product_summary = pd.read_csv('../data/processed/product_summary.csv')
    customer_summary = pd.read_csv('../data/processed/customer_summary.csv')
    
    print("Dados carregados com sucesso!")
    print(f"Vendas: {len(sales_df)} registros")
    print(f"Clientes: {len(customers_df)} registros")
    print(f"Produtos: {len(products_df)} registros")
    
except FileNotFoundError as e:
    print(f"Erro ao carregar dados: {e}")
    print("Execute o pipeline ETL primeiro!")

## 2. Análise Exploratória de Vendas

In [ ]:
# Estatísticas descritivas das vendas
print("=== Estatísticas de Vendas ===")
print(sales_df[['quantity', 'unit_price', 'total_amount']].describe())

In [ ]:
# Distribuição de vendas por categoria
plt.figure(figsize=(15, 10))

# Vendas por categoria de produto
plt.subplot(2, 2, 1)
category_sales = sales_df.groupby('category')['total_amount'].sum().sort_values(ascending=False)
category_sales.plot(kind='bar')
plt.title('Receita por Categoria de Produto')
plt.xlabel('Categoria')
plt.ylabel('Receita Total (R$)')
plt.xticks(rotation=45)

# Distribuição de valores de venda
plt.subplot(2, 2, 2)
plt.hist(sales_df['total_amount'], bins=30, alpha=0.7)
plt.title('Distribuição de Valores de Venda')
plt.xlabel('Valor da Venda (R$)')
plt.ylabel('Frequência')

# Vendas por categoria de valor
plt.subplot(2, 2, 3)
sales_df['sale_category'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribuição por Categoria de Valor')

# Vendas ao longo do tempo
plt.subplot(2, 2, 4)
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
daily_sales = sales_df.groupby(sales_df['sale_date'].dt.date)['total_amount'].sum()
daily_sales.plot()
plt.title('Vendas Diárias ao Longo do Tempo')
plt.xlabel('Data')
plt.ylabel('Receita (R$)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 3. Análise de Clientes

In [ ]:
# Análise de segmentação de clientes
plt.figure(figsize=(15, 5))

# Distribuição de clientes por segmento
plt.subplot(1, 3, 1)
customers_df['customer_segment'].value_counts().plot(kind='bar')
plt.title('Distribuição de Clientes por Segmento')
plt.xlabel('Segmento')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=45)

# Clientes por cidade
plt.subplot(1, 3, 2)
top_cities = customers_df['city'].value_counts().head(10)
top_cities.plot(kind='bar')
plt.title('Top 10 Cidades por Número de Clientes')
plt.xlabel('Cidade')
plt.ylabel('Número de Clientes')
plt.xticks(rotation=45)

# Distribuição de emails válidos
plt.subplot(1, 3, 3)
email_validity = customers_df['is_valid_email'].value_counts()
email_validity.plot(kind='pie', autopct='%1.1f%%', labels=['Email Inválido', 'Email Válido'])
plt.title('Qualidade dos Emails')

plt.tight_layout()
plt.show()

## 4. Análise de Produtos

In [ ]:
# Top produtos por vendas
plt.figure(figsize=(15, 10))

# Top 10 produtos por receita
plt.subplot(2, 2, 1)
top_products_revenue = product_summary.nlargest(10, 'total_revenue')
plt.barh(range(len(top_products_revenue)), top_products_revenue['total_revenue'])
plt.yticks(range(len(top_products_revenue)), top_products_revenue['product_name'])
plt.title('Top 10 Produtos por Receita')
plt.xlabel('Receita Total (R$)')

# Distribuição de margem de lucro
plt.subplot(2, 2, 2)
plt.hist(products_df['profit_margin'], bins=20, alpha=0.7)
plt.title('Distribuição de Margem de Lucro')
plt.xlabel('Margem de Lucro (%)')
plt.ylabel('Número de Produtos')

# Produtos por categoria de margem
plt.subplot(2, 2, 3)
products_df['margin_category'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribuição por Categoria de Margem')

# Relação preço vs quantidade vendida
plt.subplot(2, 2, 4)
product_price_qty = product_summary.merge(products_df[['product_id', 'unit_price']], on='product_id')
plt.scatter(product_price_qty['unit_price'], product_price_qty['total_quantity'], alpha=0.6)
plt.title('Preço vs Quantidade Vendida')
plt.xlabel('Preço Unitário (R$)')
plt.ylabel('Quantidade Total Vendida')

plt.tight_layout()
plt.show()

## 5. Análise Temporal

In [ ]:
# Preparar dados temporais
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
sales_df['day_name'] = sales_df['sale_date'].dt.day_name()
sales_df['hour'] = sales_df['sale_date'].dt.hour

plt.figure(figsize=(15, 10))

# Vendas por dia da semana
plt.subplot(2, 2, 1)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_sales_pattern = sales_df.groupby('day_name')['total_amount'].sum().reindex(day_order)
daily_sales_pattern.plot(kind='bar')
plt.title('Vendas por Dia da Semana')
plt.xlabel('Dia da Semana')
plt.ylabel('Receita Total (R$)')
plt.xticks(rotation=45)

# Vendas por hora do dia
plt.subplot(2, 2, 2)
hourly_sales = sales_df.groupby('hour')['total_amount'].sum()
hourly_sales.plot()
plt.title('Vendas por Hora do Dia')
plt.xlabel('Hora')
plt.ylabel('Receita Total (R$)')

# Vendas mensais
plt.subplot(2, 2, 3)
monthly_sales = sales_df.groupby(sales_df['sale_date'].dt.to_period('M'))['total_amount'].sum()
monthly_sales.plot(kind='bar')
plt.title('Vendas Mensais')
plt.xlabel('Mês')
plt.ylabel('Receita Total (R$)')
plt.xticks(rotation=45)

# Trend de vendas
plt.subplot(2, 2, 4)
daily_trend = sales_df.groupby(sales_df['sale_date'].dt.date)['total_amount'].sum()
daily_trend.plot()
# Adicionar linha de tendência
z = np.polyfit(range(len(daily_trend)), daily_trend.values, 1)
p = np.poly1d(z)
plt.plot(daily_trend.index, p(range(len(daily_trend))), "r--", alpha=0.8)
plt.title('Tendência de Vendas Diárias')
plt.xlabel('Data')
plt.ylabel('Receita (R$)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 6. KPIs e Métricas de Negócio

In [ ]:
# Calcular KPIs principais
total_revenue = sales_df['total_amount'].sum()
total_orders = len(sales_df)
unique_customers = sales_df['customer_id'].nunique()
avg_order_value = total_revenue / total_orders
revenue_per_customer = total_revenue / unique_customers

print("=== KPIs Principais ===")
print(f"Receita Total: R$ {total_revenue:,.2f}")
print(f"Total de Pedidos: {total_orders:,}")
print(f"Clientes Únicos: {unique_customers:,}")
print(f"Ticket Médio: R$ {avg_order_value:,.2f}")
print(f"Receita por Cliente: R$ {revenue_per_customer:,.2f}")

# Análise de cohorte simplificada
customer_first_purchase = sales_df.groupby('customer_id')['sale_date'].min().reset_index()
customer_first_purchase.columns = ['customer_id', 'first_purchase_date']

# Mesclar com dados de vendas
sales_with_cohort = sales_df.merge(customer_first_purchase, on='customer_id')
sales_with_cohort['first_purchase_month'] = sales_with_cohort['first_purchase_date'].dt.to_period('M')
sales_with_cohort['purchase_month'] = sales_with_cohort['sale_date'].dt.to_period('M')

print("\n=== Análise de Cohorte ===")
cohort_sizes = sales_with_cohort.groupby('first_purchase_month')['customer_id'].nunique()
print("Tamanho das cohortes por mês de primeira compra:")
print(cohort_sizes)

## 7. Insights e Recomendações

In [ ]:
print("=== INSIGHTS E RECOMENDAÇÕES ===")
print()

# Análise de categorias
best_category = category_sales.index[0]
best_category_revenue = category_sales.iloc[0]
print(f"1. CATEGORIA LÍDER: {best_category}")
print(f"   - Receita: R$ {best_category_revenue:,.2f}")
print(f"   - Representa {(best_category_revenue/total_revenue)*100:.1f}% da receita total")
print()

# Análise de produtos
best_product = top_products_revenue.iloc[0]
print(f"2. PRODUTO MAIS VENDIDO: {best_product['product_name']}")
print(f"   - Receita: R$ {best_product['total_revenue']:,.2f}")
print(f"   - Vendas: {best_product['total_sales']} unidades")
print()

# Análise de clientes
active_customers = customers_df[customers_df['customer_segment'] == 'Active'].shape[0]
total_customers = len(customers_df)
print(f"3. SEGMENTAÇÃO DE CLIENTES:")
print(f"   - Clientes ativos: {active_customers} ({(active_customers/total_customers)*100:.1f}%)")
print(f"   - Oportunidade de reativação para clientes inativos")
print()

# Análise temporal
best_day = daily_sales_pattern.idxmax()
print(f"4. PADRÃO TEMPORAL:")
print(f"   - Melhor dia da semana: {best_day}")
print(f"   - Recomendação: Concentrar campanhas de marketing neste dia")
print()

print("=== PRÓXIMOS PASSOS SUGERIDOS ===")
print("1. Implementar análise de cesta de mercado")
print("2. Desenvolver modelo de recomendação de produtos")
print("3. Criar campanhas direcionadas por segmento de cliente")
print("4. Monitorar métricas de retenção de clientes")
print("5. Análise de sazonalidade com mais dados históricos")